In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
from pinecone import Pinecone, ServerlessSpec

In [4]:
os.environ['PINECONE_API_KEY'] = "fc9fbc45-7ff2-4e20-8dab-4b59fc443682"

In [5]:
pc = Pinecone()
pc.create_index(
  name="rag", 
  dimension=1536, 
  metric="cosine",
  spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )
